In [ ]:
!pip install ta scikit-learn tensorflow


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ba61cfd256ac0f0370e100a028c6457e2812b2f92197a38f1751512e67ff3b76
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving HDFCBANK.csv to HDFCBANK.csv


In [ ]:
# Load CSV
df = pd.read_csv("HDFCBANK.csv")
df.columns = [col.lower() for col in df.columns]
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Clean close price
df['close'] = pd.to_numeric(df['close'], errors='coerce')
df.dropna(subset=['close'], inplace=True)

# Add indicators
df['rsi'] = RSIIndicator(close=df['close'], window=14).rsi()
macd = MACD(close=df['close'])
df['macd'] = macd.macd()
bb = BollingerBands(close=df['close'])
df['bb_upper'] = bb.bollinger_hband()
df['bb_lower'] = bb.bollinger_lband()
df['bb_mavg'] = bb.bollinger_mavg()

# Target: 1 if price goes up next day, else 0
df['target'] = (df['close'].shift(-1) > df['close']).astype(int)

# Drop NAs
df.dropna(inplace=True)


In [ ]:
# Select features
features = ['close', 'rsi', 'macd', 'bb_upper', 'bb_lower', 'bb_mavg']
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])

# Create sequences
X = []
y = []
window = 60  # 60-day window

for i in range(window, len(df)):
    X.append(scaled[i - window:i])
    y.append(df['target'].iloc[i])

X, y = np.array(X), np.array(y)


In [ ]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.1, callbacks=[es], verbose=1)


Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.4924 - loss: 0.6976 - val_accuracy: 0.4905 - val_loss: 0.6936
Epoch 2/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.5359 - loss: 0.6930 - val_accuracy: 0.4857 - val_loss: 0.6940
Epoch 3/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.5097 - loss: 0.6936 - val_accuracy: 0.5143 - val_loss: 0.6933
Epoch 4/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.5288 - loss: 0.6918 - val_accuracy: 0.4619 - val_loss: 0.6937
Epoch 5/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.5014 - loss: 0.6934 - val_accuracy: 0.5143 - val_loss: 0.6936
Epoch 6/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - accuracy: 0.4989 - loss: 0.6961 - val_accuracy: 0.5143 - val_loss: 0.6934
Epoch 7/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.5148 - loss: 0.6927 - val_accuracy: 0.4952 - val_loss: 0.6936
Epoch 8/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.5024 - loss: 0.6928 - val_accuracy: 0.4952 - val_loss: 0

In [ ]:
y_pred_proba = model.predict(X_test).flatten()
y_pred = (y_pred_proba > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\n📊 LSTM Performance for HDFC Bank:")
print(f"Accuracy        : {accuracy:.2f}")
print(f"Precision       : {precision:.2f}")
print(f"Recall          : {recall:.2f}")
print(f"F1 Score        : {f1:.2f}")
print(f"ROC AUC Score   : {roc_auc:.2f}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step

📊 LSTM Performance for HDFC Bank:
Accuracy        : 0.53
Precision       : 0.53
Recall          : 0.88
F1 Score        : 0.66
ROC AUC Score   : 0.48


In [ ]:
# Align predictions with actual price
test_df = df.iloc[-len(y_test):].copy()
test_df['pred'] = y_pred
test_df['return'] = test_df['close'].pct_change()
test_df['position'] = test_df['pred'].replace({0: -1, 1: 1})
test_df['strategy'] = test_df['position'].shift(1) * test_df['return']

# Sharpe & Drawdown
sharpe = (test_df['strategy'].mean() / test_df['strategy'].std()) * np.sqrt(252)
cumulative = (1 + test_df['strategy']).cumprod()
drawdown = (cumulative.cummax() - cumulative) / cumulative.cummax()
max_drawdown = drawdown.max()

print(f"Sharpe Ratio     : {sharpe:.2f}")
print(f"Max Drawdown (%) : {max_drawdown * 100:.2f}%")


Sharpe Ratio     : 0.25
Max Drawdown (%) : 22.91%


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving BAJFINANCE.csv to BAJFINANCE.csv
Saving BHARTIARTL.csv to BHARTIARTL.csv
Saving HDFCBANK.csv to HDFCBANK (1).csv
Saving INFY.csv to INFY.csv
Saving ITC.csv to ITC.csv
Saving LT.csv to LT.csv
Saving RELIANCE.csv to RELIANCE.csv
Saving SUNPHARMA.csv to SUNPHARMA.csv
Saving TATACHEM.csv to TATACHEM.csv
Saving TATAMOTORS.csv to TATAMOTORS.csv


In [ ]:
stock_files = {
    'HDFC Bank': 'HDFCBANK.csv',
    'Reliance': 'RELIANCE.csv',
    'Infosys': 'INFY.csv',
    'ITC': 'ITC.csv',
    'Tata Motors': 'TATAMOTORS.csv',
    'Sun Pharma': 'SUNPHARMA.csv',
    'L&T': 'LT.csv',
    'Bharti Airtel': 'BHARTIARTL.csv',
    'Bajaj Finance': 'BAJFINANCE.csv',
    'Tata Chemicals': 'TATACHEM.csv'
}


In [ ]:
!pip install ta scikit-learn tensorflow

import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

stock_files = {
    'HDFC Bank': 'HDFCBANK.csv',
    'Reliance': 'RELIANCE.csv',
    'Infosys': 'INFY.csv',
    'ITC': 'ITC.csv',
    'Tata Motors': 'TATAMOTORS.csv',
    'Sun Pharma': 'SUNPHARMA.csv',
    'L&T': 'LT.csv',
    'Bharti Airtel': 'BHARTIARTL.csv',
    'Bajaj Finance': 'BAJFINANCE.csv',
    'Tata Chemicals': 'TATACHEM.csv'
}

results = []

for stock_name, file_path in stock_files.items():
    try:
        # Load & clean
        df = pd.read_csv(file_path)
        df.columns = [col.lower() for col in df.columns]
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df.set_index('date', inplace=True)
        df['close'] = pd.to_numeric(df['close'], errors='coerce')
        df.dropna(subset=['close'], inplace=True)

        # Indicators
        df['rsi'] = RSIIndicator(close=df['close'], window=14).rsi()
        macd = MACD(close=df['close'])
        df['macd'] = macd.macd()
        bb = BollingerBands(close=df['close'])
        df['bb_upper'] = bb.bollinger_hband()
        df['bb_lower'] = bb.bollinger_lband()
        df['bb_mavg'] = bb.bollinger_mavg()

        # Target
        df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
        df.dropna(inplace=True)

        # Features
        features = ['close', 'rsi', 'macd', 'bb_upper', 'bb_lower', 'bb_mavg']
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(df[features])

        # Sequences
        X, y = [], []
        window = 60
        for i in range(window, len(df)):
            X.append(scaled[i-window:i])
            y.append(df['target'].iloc[i])
        X, y = np.array(X), np.array(y)

        # Train/test
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(0.2))
        model.add(LSTM(50))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.1, callbacks=[es], verbose=0)

        # Predict
        y_proba = model.predict(X_test).flatten()
        y_pred = (y_proba > 0.5).astype(int)

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        # Strategy
        test_df = df.iloc[-len(y_test):].copy()
        test_df['pred'] = y_pred
        test_df['return'] = test_df['close'].pct_change()
        test_df['position'] = test_df['pred'].replace({0: -1, 1: 1})
        test_df['strategy'] = test_df['position'].shift(1) * test_df['return']
        test_df.dropna(inplace=True)

        sharpe = (test_df['strategy'].mean() / test_df['strategy'].std()) * np.sqrt(252) if test_df['strategy'].std() != 0 else 0
        cum = (1 + test_df['strategy']).cumprod()
        dd = (cum.cummax() - cum) / cum.cummax()
        max_dd = dd.max()

        results.append({
            'Stock': stock_name,
            'Accuracy': round(acc, 2),
            'Precision': round(prec, 2),
            'Recall': round(rec, 2),
            'F1 Score': round(f1, 2),
            'ROC AUC': round(roc_auc, 2),
            'Sharpe Ratio': round(sharpe, 2),
            'Max Drawdown (%)': round(max_dd * 100, 2)
        })

        print(f"✅ {stock_name} done")

    except Exception as e:
        print(f"❌ {stock_name} failed: {e}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
✅ HDFC Bank done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
✅ Reliance done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
✅ Infosys done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
✅ ITC done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
✅ Tata Motors done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
✅ Sun Pharma done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
✅ L&T done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
✅ Bharti Airtel done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
✅ Bajaj Finance done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
✅ Tata Chemicals done


In [ ]:
results_df = pd.DataFrame(results)
results_df


,Stock,Accuracy,Precision,Recall,F1 Score,ROC AUC,Sharpe Ratio,Max Drawdown (%)
0,HDFC Bank,0.53,0.53,1.00,0.70,0.46,0.40,19.91
1,Reliance,0.50,0.50,1.00,0.67,0.51,0.12,24.46
2,Infosys,0.50,0.50,1.00,0.66,0.50,0.61,26.17
3,ITC,0.51,0.51,0.90,0.65,0.53,1.18,16.84
4,Tata Motors,0.50,0.00,0.00,0.00,0.53,-1.09,70.95
5,Sun Pharma,0.53,0.54,0.79,0.64,0.50,1.20,18.88
6,L&T,0.46,0.00,0.00,0.00,0.53,-1.33,55.73
7,Bharti Airtel,0.52,0.52,1.00,0.68,0.51,1.69,13.89
8,Bajaj Finance,0.49,0.49,1.00,0.66,0.50,0.10,22.74
9,Tata Chemicals,0.53,0.53,1.00,0.69,0.49,0.15,24.20


In [ ]:
# Save LSTM results
lstm_df = pd.DataFrame(results)  # Assuming your LSTM results are in `results`
lstm_df.to_csv('lstm_results.csv', index=False)

# Download
from google.colab import files
files.download('lstm_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>